In [8]:
import pandas as pd
import datetime as dt
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
import sys
sys.path.insert(0, '/Users/acheung/TC_RI_P1/scripts/Useful Functions/')
from distance import distance_calculator
from cylindrical_conversion import interp_to_equal_grid
from scipy import ndimage
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


In [9]:
# Determine datetimes of Storm ATCF ID Desired

storm_ID = 'AL082021'

SHIPS = pd.read_csv('/Users/acheung/data/SHIPS/all_SHIPs_data_combined_north_atlantic.csv')

SHIPS['Time'] = pd.to_datetime(SHIPS['Time'])

current_storm_SHIPS = SHIPS.where(SHIPS['Storm_ID'] == storm_ID).dropna()
time_begin = current_storm_SHIPS['Time'].min()
time_end = current_storm_SHIPS['Time'].max()
times_pd = pd.date_range(time_begin, time_end, freq="30min")


In [10]:
filepaths = []
for i in times_pd: # Download all the desired IMERG Images
    # If IMERG file does not exist, download it
    
    year = i.year
    day_of_year = i.strftime('%j')
    day_stripped = i.strftime('%Y%m%d')
    time_stripped = i.strftime('%H%M%S')
    min_of_day = str(i.hour*60 + i.minute).zfill(4)
    end_time_stripped = (i+dt.timedelta(minutes=29,seconds=59)).strftime('%H%M%S')
        
    IMERG_file_path = '/Users/acheung/data/IMERG/'+str(year)+'/'+day_of_year+'/'+\
            '3B-HHR.MS.MRG.3IMERG.'+day_stripped+'-S'+time_stripped+'-E'+end_time_stripped+\
            '.'+min_of_day+'.V07B.HDF5'  

# If day directory does not exist, make day directory
    if os.path.exists('/Users/acheung/data/IMERG/'+str(year)) == False:
        os.mkdir('/Users/acheung/data/IMERG/'+str(year))

    if os.path.exists('/Users/acheung/data/IMERG/'+str(year)+'/'+day_of_year) == False:
        os.mkdir('/Users/acheung/data/IMERG/'+str(year)+'/'+day_of_year)

    if os.path.exists(IMERG_file_path) == False:
        url_desired = "https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.07/"\
            +str(year)+'/'+day_of_year+'/'+'3B-HHR.MS.MRG.3IMERG.'+day_stripped+'-S'+time_stripped+\
            '-E'+end_time_stripped+'.'+min_of_day+'.V07B.HDF5'
        os.chdir('/Users/acheung/data/IMERG/'+str(year)+'/'+day_of_year)

        os.system('wget --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --keep-session-cookies '
                  + url_desired)
    filepaths.append(IMERG_file_path)


--2024-09-13 15:10:38--  https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.07/2021/227/3B-HHR.MS.MRG.3IMERG.20210815-S190000-E192959.1140.V07B.HDF5
Resolving gpm1.gesdisc.eosdis.nasa.gov (gpm1.gesdisc.eosdis.nasa.gov)... 198.118.197.50, 2001:4d0:241a:4041::50
Connecting to gpm1.gesdisc.eosdis.nasa.gov (gpm1.gesdisc.eosdis.nasa.gov)|198.118.197.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7849582 (7.5M) [application/x-troff-ms]
Saving to: ‘3B-HHR.MS.MRG.3IMERG.20210815-S190000-E192959.1140.V07B.HDF5’

     0K .......... .......... .......... .......... ..........  0% 9.18M 1s
    50K .......... .......... .......... .......... ..........  1% 18.0M 1s
   100K .......... .......... .......... .......... ..........  1% 10.3M 1s
   150K .......... .......... .......... .......... ..........  2% 12.5M 1s
   200K .......... .......... .......... .......... ..........  3% 14.5M 1s
   250K .......... .......... .......... .......... ..........  3% 17.

In [11]:
all_last_slices = []
for case_loop in range(len(filepaths)): # Run all
# for case_loop in [18,34,54]: # Run all
    fn = filepaths[case_loop]
    print(fn)
    try:
        f = h5py.File(fn, 'r')
    except: # If the file is corrupt, remove and re-download
        os.system('rm ' + fn)
        os.chdir(fn[0:35])
        new_url = 'https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.07/' + fn[26:]
        os.system('wget --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --keep-session-cookies '
                  + new_url)
        f = h5py.File(fn, 'r')
        
    # Work on precip file
    groups = [ x for x in f.keys() ]
    # print(groups)
    gridMembers = [ x for x in f['Grid'] ]
    # print(gridMembers)

    # Get the precipitation, latitude, and longitude variables
    precip = f['Grid/precipitation'][0][:][:]
    precip = np.transpose(precip)
    precip[precip<-999]=np.nan

    theLats = f['Grid/lat'][:]
    theLons = f['Grid/lon'][:]
    x, y = np.float32(np.meshgrid(theLons, theLats))

    time_float_arr = np.asarray(current_storm_SHIPS['Time']).astype(float)
    bt_lats = current_storm_SHIPS['LAT']
    bt_lons = -current_storm_SHIPS['LON']
    
    if len(bt_lats) > 0:
        latspl = UnivariateSpline(time_float_arr,bt_lats,k=2,s=0)
        centering_lat = latspl(np.asarray(times_pd).astype(float)[case_loop])
    if len(bt_lons) > 0:
        lonspl = UnivariateSpline(time_float_arr,bt_lons,k=2,s=0)
        centering_lon = lonspl(np.asarray(times_pd).astype(float)[case_loop])
        
    # Find index closest to interpolated best-track center or 2-km radar center
    distance_arr = distance_calculator(x, y,(centering_lon,centering_lat))

    abs_dist_arr = (abs(distance_arr))

    min_dist_ind = np.where(abs_dist_arr == np.nanmin(abs_dist_arr))
    
    # Interpolate SHIPS
    
    bt_shear_dirs = current_storm_SHIPS['SHTD']
    shearspl = UnivariateSpline(time_float_arr,bt_shear_dirs,k=2,s=0)
    interp_shear = shearspl(np.asarray(times_pd).astype(float)[case_loop])

    angle_rotate = 90 - interp_shear
    
    # NEVER FORGET TO MULTIPLY RADIUS ARRAYS BY DX FOR IMERG PLOTS!!!

    dx = 2
    dy = 2

    # Interpolate IMERG data to equal-distance grid
    
    # Slice arrays to within 100 indices of interpolated best-track center

    sliced_lon_grid = x[min_dist_ind[0][0]-70:min_dist_ind[0][0]+70,min_dist_ind[1][0]-70:min_dist_ind[1][0]+70]

    sliced_lat_grid = y[min_dist_ind[0][0]-70:min_dist_ind[0][0]+70,min_dist_ind[1][0]-70:min_dist_ind[1][0]+70]

    sliced_precip = precip[min_dist_ind[0][0]-70:min_dist_ind[0][0]+70,min_dist_ind[1][0]-70:min_dist_ind[1][0]+70]

    eq_lon_grid,eq_lat_grid,eq_dist_data = interp_to_equal_grid(sliced_lon_grid,sliced_lat_grid,
                                                            sliced_precip,dx = dx,dy=dy)
    
    distance_arr_sliced = distance_calculator(eq_lon_grid, eq_lat_grid,(centering_lon,centering_lat))

    abs_dist_arr_sliced = (abs(distance_arr_sliced))

    min_dist_ind_sliced = np.where(abs_dist_arr_sliced == np.nanmin(abs_dist_arr_sliced))

    pts_above = eq_dist_data.shape[0] - min_dist_ind_sliced[0][0]
    y_range = np.arange(-min_dist_ind_sliced[0][0],pts_above) * dy

    pts_right = eq_dist_data.shape[1] - min_dist_ind_sliced[1][0]
    x_range = np.arange(-min_dist_ind_sliced[1][0],pts_right) * dx

    xv, yv = np.meshgrid(x_range, y_range)

    x_range.shape[0],np.where(x_range==0)[0][0]

    x_zero_loc = np.where(x_range==0)[0][0]

    if x_zero_loc > x_range.shape[0]/2:
        x_offset = int(x_range.shape[0]-x_zero_loc)
    elif x_zero_loc <= x_range.shape[0]/2:
        x_offset = x_zero_loc

    y_zero_loc = np.where(y_range==0)[0][0]

    if y_zero_loc > y_range.shape[0]/2:
        y_offset = int(y_range.shape[0]-y_zero_loc)
    elif y_zero_loc <= y_range.shape[0]/2:
        y_offset = y_zero_loc

    x_even_inds = x_range[x_zero_loc-(x_offset-1):x_zero_loc+x_offset+1]
    y_even_inds = y_range[y_zero_loc-(y_offset-1):y_zero_loc+y_offset+1]

    sliced_eq_data_pd = pd.DataFrame(eq_dist_data,columns=x_range,index=y_range)

    evenly_sliced_pd = sliced_eq_data_pd[x_even_inds].loc[y_even_inds]

    x_evenly_sliced, y_evenly_sliced = np.meshgrid(x_even_inds, y_even_inds)
        
    img_rotate = ndimage.rotate(evenly_sliced_pd, angle_rotate, reshape=False,cval=np.nan,prefilter=False)
    img_rotate_pd = pd.DataFrame(img_rotate,columns=evenly_sliced_pd.columns,index=evenly_sliced_pd.index)

    pd_last_slice = img_rotate_pd[np.arange(-300,300.1,dx)].loc[-300:300]
    x_evenly_last_sliced, y_evenly_last_sliced = np.meshgrid(pd_last_slice.columns, pd_last_slice.index)
    
    # Save rotated and sliced data to list
    # all_last_slices.append(pd_last_slice)
    plt.figure(figsize=(10,7))
    ax = plt.axes(adjustable='box', aspect=1)
    now_plot = ax.contourf(pd_last_slice,levels=np.arange(0,40.1,4),cmap='jet')
    plt.colorbar(now_plot,label='Precipitation (mm/h)')
    ax.grid()
    ax.set_title(current_storm_SHIPS['Name'].unique()[0] + ' ' + str(times_pd[case_loop]))
    axins = inset_axes(ax, width="25%", height="9%", loc=3, borderpad=0)
    axins.tick_params(labelleft=False, labelbottom=False,left = False,bottom = False)
    axins.patch.set_alpha(0.6)
    q = axins.arrow(0,0,0.05,0,color='black',head_length = 0.009)
    ax.set_xlabel('Along-Shear Direction (km)')
    ax.set_ylabel('Across-Shear Direction (km)')
    plt.savefig('/Users/acheung/data/Figures/Individual_Storms_IMERG/intermediates/'+current_storm_SHIPS['Name'].unique()[0] + '_' +
                str(times_pd[case_loop]) + '.png')
    plt.close()



/Users/acheung/data/IMERG/2021/227/3B-HHR.MS.MRG.3IMERG.20210815-S180000-E182959.1080.V07B.HDF5
/Users/acheung/data/IMERG/2021/227/3B-HHR.MS.MRG.3IMERG.20210815-S183000-E185959.1110.V07B.HDF5
/Users/acheung/data/IMERG/2021/227/3B-HHR.MS.MRG.3IMERG.20210815-S190000-E192959.1140.V07B.HDF5
/Users/acheung/data/IMERG/2021/227/3B-HHR.MS.MRG.3IMERG.20210815-S193000-E195959.1170.V07B.HDF5
/Users/acheung/data/IMERG/2021/227/3B-HHR.MS.MRG.3IMERG.20210815-S200000-E202959.1200.V07B.HDF5
/Users/acheung/data/IMERG/2021/227/3B-HHR.MS.MRG.3IMERG.20210815-S203000-E205959.1230.V07B.HDF5
/Users/acheung/data/IMERG/2021/227/3B-HHR.MS.MRG.3IMERG.20210815-S210000-E212959.1260.V07B.HDF5
/Users/acheung/data/IMERG/2021/227/3B-HHR.MS.MRG.3IMERG.20210815-S213000-E215959.1290.V07B.HDF5
/Users/acheung/data/IMERG/2021/227/3B-HHR.MS.MRG.3IMERG.20210815-S220000-E222959.1320.V07B.HDF5
/Users/acheung/data/IMERG/2021/227/3B-HHR.MS.MRG.3IMERG.20210815-S223000-E225959.1350.V07B.HDF5
/Users/acheung/data/IMERG/2021/227/3B-HH

/Users/acheung/TC_RI_P1/scripts/Useful Functions/distance.py:25: RuntimeWarning: invalid value encountered in sqrt
  c = 2. * np.arctan2(np.sqrt(a), np.sqrt(1.-a))


/Users/acheung/data/IMERG/2021/230/3B-HHR.MS.MRG.3IMERG.20210818-S170000-E172959.1020.V07B.HDF5
/Users/acheung/data/IMERG/2021/230/3B-HHR.MS.MRG.3IMERG.20210818-S173000-E175959.1050.V07B.HDF5
/Users/acheung/data/IMERG/2021/230/3B-HHR.MS.MRG.3IMERG.20210818-S180000-E182959.1080.V07B.HDF5
/Users/acheung/data/IMERG/2021/230/3B-HHR.MS.MRG.3IMERG.20210818-S183000-E185959.1110.V07B.HDF5
/Users/acheung/data/IMERG/2021/230/3B-HHR.MS.MRG.3IMERG.20210818-S190000-E192959.1140.V07B.HDF5
/Users/acheung/data/IMERG/2021/230/3B-HHR.MS.MRG.3IMERG.20210818-S193000-E195959.1170.V07B.HDF5
/Users/acheung/data/IMERG/2021/230/3B-HHR.MS.MRG.3IMERG.20210818-S200000-E202959.1200.V07B.HDF5
/Users/acheung/data/IMERG/2021/230/3B-HHR.MS.MRG.3IMERG.20210818-S203000-E205959.1230.V07B.HDF5
/Users/acheung/data/IMERG/2021/230/3B-HHR.MS.MRG.3IMERG.20210818-S210000-E212959.1260.V07B.HDF5
/Users/acheung/data/IMERG/2021/230/3B-HHR.MS.MRG.3IMERG.20210818-S213000-E215959.1290.V07B.HDF5
/Users/acheung/data/IMERG/2021/230/3B-HH

/Users/acheung/TC_RI_P1/scripts/Useful Functions/distance.py:25: RuntimeWarning: invalid value encountered in sqrt
  c = 2. * np.arctan2(np.sqrt(a), np.sqrt(1.-a))


/Users/acheung/data/IMERG/2021/233/3B-HHR.MS.MRG.3IMERG.20210821-S193000-E195959.1170.V07B.HDF5
/Users/acheung/data/IMERG/2021/233/3B-HHR.MS.MRG.3IMERG.20210821-S200000-E202959.1200.V07B.HDF5
/Users/acheung/data/IMERG/2021/233/3B-HHR.MS.MRG.3IMERG.20210821-S203000-E205959.1230.V07B.HDF5
/Users/acheung/data/IMERG/2021/233/3B-HHR.MS.MRG.3IMERG.20210821-S210000-E212959.1260.V07B.HDF5
/Users/acheung/data/IMERG/2021/233/3B-HHR.MS.MRG.3IMERG.20210821-S213000-E215959.1290.V07B.HDF5
/Users/acheung/data/IMERG/2021/233/3B-HHR.MS.MRG.3IMERG.20210821-S220000-E222959.1320.V07B.HDF5
/Users/acheung/data/IMERG/2021/233/3B-HHR.MS.MRG.3IMERG.20210821-S223000-E225959.1350.V07B.HDF5
/Users/acheung/data/IMERG/2021/233/3B-HHR.MS.MRG.3IMERG.20210821-S230000-E232959.1380.V07B.HDF5
/Users/acheung/data/IMERG/2021/233/3B-HHR.MS.MRG.3IMERG.20210821-S233000-E235959.1410.V07B.HDF5
/Users/acheung/data/IMERG/2021/234/3B-HHR.MS.MRG.3IMERG.20210822-S000000-E002959.0000.V07B.HDF5
/Users/acheung/data/IMERG/2021/234/3B-HH

/Users/acheung/TC_RI_P1/scripts/Useful Functions/distance.py:25: RuntimeWarning: invalid value encountered in sqrt
  c = 2. * np.arctan2(np.sqrt(a), np.sqrt(1.-a))


/Users/acheung/data/IMERG/2021/234/3B-HHR.MS.MRG.3IMERG.20210822-S053000-E055959.0330.V07B.HDF5
/Users/acheung/data/IMERG/2021/234/3B-HHR.MS.MRG.3IMERG.20210822-S060000-E062959.0360.V07B.HDF5
/Users/acheung/data/IMERG/2021/234/3B-HHR.MS.MRG.3IMERG.20210822-S063000-E065959.0390.V07B.HDF5
/Users/acheung/data/IMERG/2021/234/3B-HHR.MS.MRG.3IMERG.20210822-S070000-E072959.0420.V07B.HDF5
/Users/acheung/data/IMERG/2021/234/3B-HHR.MS.MRG.3IMERG.20210822-S073000-E075959.0450.V07B.HDF5
/Users/acheung/data/IMERG/2021/234/3B-HHR.MS.MRG.3IMERG.20210822-S080000-E082959.0480.V07B.HDF5
/Users/acheung/data/IMERG/2021/234/3B-HHR.MS.MRG.3IMERG.20210822-S083000-E085959.0510.V07B.HDF5
/Users/acheung/data/IMERG/2021/234/3B-HHR.MS.MRG.3IMERG.20210822-S090000-E092959.0540.V07B.HDF5
/Users/acheung/data/IMERG/2021/234/3B-HHR.MS.MRG.3IMERG.20210822-S093000-E095959.0570.V07B.HDF5
/Users/acheung/data/IMERG/2021/234/3B-HHR.MS.MRG.3IMERG.20210822-S100000-E102959.0600.V07B.HDF5
/Users/acheung/data/IMERG/2021/234/3B-HH

In [12]:
pd_last_slice

,-300,-298,-296,-294,-292,-290,-288,-286,-284,-282,...,282,284,286,288,290,292,294,296,298,300
-300,0.0,0.0,0.0,0.0,0.0,1.309571e-05,0.000524,0.002707,0.006435,0.010548,...,0.000823,0.003839,0.008366,0.013017,0.017244,0.019493,0.019491,0.019375,0.019865,0.033989
-298,0.0,0.0,0.0,0.0,0.0,7.300199e-06,0.000609,0.003325,0.008171,0.013502,...,0.000741,0.003744,0.008092,0.012525,0.016503,0.018855,0.019556,0.020270,0.022291,0.035255
-296,0.0,0.0,0.0,0.0,0.0,2.160530e-06,0.000631,0.003752,0.009497,0.015739,...,0.000573,0.003311,0.007084,0.010822,0.014199,0.016731,0.018699,0.020916,0.024611,0.036746
-294,0.0,0.0,0.0,0.0,0.0,8.248350e-08,0.000531,0.003527,0.009106,0.015030,...,0.000369,0.002598,0.005595,0.008492,0.011227,0.014176,0.017609,0.021496,0.026548,0.037785
-292,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000350,0.002736,0.007263,0.012064,...,0.000223,0.001874,0.004100,0.006164,0.008190,0.011601,0.016501,0.022057,0.028381,0.038373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
294,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
296,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
298,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
out_file = current_storm_SHIPS['Name'].unique()[0] + str(times_pd[-1].year) + '.gif'
os.chdir('/Users/acheung/data/Figures/Individual_Storms_IMERG/intermediates/')
os.system('convert -delay 20 -loop 0 *.png /Users/acheung/data/Figures/Individual_Storms_IMERG/GIFs/'+out_file)
os.system('rm -r *.png')

0